In [4]:
#import shutil as shutil
#shutil.which("python3")


In [5]:
from pathlib import Path
from bs4 import BeautifulSoup
import re as re

#current_path = Path(__file__).parent
file_path = "/home/aurelien/Programation/telegramSA/Data/messages.html"


with open(file_path) as fp:
    soup = BeautifulSoup(fp, "lxml")

In [1]:
#soup.contents

In [2]:
#print(soup.find(attrs={"class" : "message default clearfix"}).prettify())

In [6]:
raw_messages = soup.find_all(attrs = {"class" : "message default clearfix"})

def findMedia(html_msg):
    ret = None
    if html_msg.find(attrs = {"class" : "media"}):
        ret = html_msg
    return ret

media_lst = [ findMedia(media) for media in raw_messages]
#media_lst

In [9]:
raw_messages = soup.find_all(attrs = {"class" : "message default clearfix"})

def cleanHashtags(html_message_list):
    try:
        ret = [hashtag.string.strip("#") for hashtag in html_message_list]
    except:
        ret = html_message_list
    return ret

def cleanLinks(html_links_list):
    ret = [link["href"] for link in html_links_list]
    return ret

def extractMessage(html_message):
    msg = { "message_id" : html_message['id'],
            "time" : html_message.find(attrs = {"class" : "date"})['title'],
            "user" : html_message.find(attrs = {"class" : "from_name"}).string,
            "message" :  None,
            "hashtag" : cleanHashtags(html_message.find_all(attrs = {"onclick" : re.compile("ShowHashtag(.*)")})),
            "links" : cleanLinks(html_message.find_all(href=re.compile("."))),
            "raw_message" : html_message
    }
    if html_message.find(attrs = {"class" : "text"}):
        msg["message"] = html_message.find(attrs = {"class" : "text"}).next_element
        # using the fact that .next_element return the inside of the message which is empty before 
        # the next <a> tag inside (for cases of link/hashtags)
    else:
        msg["message"] = "not found"
    try:
        msg["user"] = msg["user"].strip()
        msg["message"] = msg["message"].strip()
    except:
        pass
    return msg



msg_list = [extractMessage(items) for items in raw_messages]
msg_list[3]

#for l in msg_list:
#    if l["message"] == "":
#        print("###########")
#        print(l["raw_message"])

{'message_id': 'message173666',
 'time': '02.06.2019 21:16:08',
 'user': 'Aurélien',
 'message': 'Ok, I just parked home, I will go up to the flat now',
 'hashtag': [],
 'links': [],
 'raw_message': <div class="message default clearfix" id="message173666">
 <div class="pull_left userpic_wrap">
 <div class="userpic userpic5" style="width: 42px; height: 42px">
 <div class="initials" style="line-height: 42px">
 A
         </div>
 </div>
 </div>
 <div class="body">
 <div class="pull_right date details" title="02.06.2019 21:16:08">
 21:16
        </div>
 <div class="from_name">
 Aurélien 
        </div>
 <div class="text">
 Ok, I just parked home, I will go up to the flat now
        </div>
 </div>
 </div>}